In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
from quantfreedom.levon_qf.strategy_maker_cls_levon import StrategyMaker

import numpy as np
import pandas as pd

from quantfreedom.data.data_dl import data_download_from_ccxt
from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    CandleBody,
)
from quantfreedom.plotting.strat_dashboard import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )
sm = StrategyMaker()
# prices.to_hdf("yt_prices.hd5", key='data')
price_data = pd.read_hdf('../tests/data/prices.hd5')
rsi_ind = sm.from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = rsi_ind.is_below(
    user_args=[40, 50],
    cand_ohlc="close",
)
sm.combined_data_frame()

symbol              BTCUSDT                      EOSUSD                       \
RSI_timeperiod           15            30            15            30          
RSI_is_below             40     50     40     50     40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00   False  False  False  False  False  False  False  False   
2020-10-01 00:30:00   False  False  False  False  False  False  False  False   
2020-10-01 01:00:00   False  False  False  False  False  False  False  False   
2020-10-01 01:30:00   False  False  False  False  False  False  False  False   
2020-10-01 02:00:00   False  False  False  False  False  False  False  False   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
2021-04-14 21:30:00   False   True  False   True  False  False  False  False   
2021-04-14 22:00:00   False   True  False   True  False  False  False  False   
2021-04-14 22:30:00   False   True  False   True  False  False  False  False   
2021-04-14 23:00:00   False   True  False   True  False  False  False  False   
2021-04-14 23:30:00   False   True  False   True  False  False  False  False   

symbol              ETHUSD                      LTCUSDT                       \
RSI_timeperiod          15            30             15            30          
RSI_is_below            40     50     40     50      40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00  False  False  False  False   False  False  False  False   
2020-10-01 00:30:00  False  False  False  False   False  False  False  False   
2020-10-01 01:00:00  False  False  False  False   False  False  False  False   
2020-10-01 01:30:00  False  False  False  False   False  False  False  False   
2020-10-01 02:00:00  False  False  False  False   False  False  False  False   
...                    ...    ...    ...    ...     ...    ...    ...    ...   
2021-04-14 21:30:00  False  False  False  False   False  False  False  False   
2021-04-14 22:00:00  False  False  False  False   False  False  False  False   
2021-04-14 22:30:00  False  False  False  False   False  False  False  False   
2021-04-14 23:00:00  False  False  False  False   False  False  False  False   
2021-04-14 23:30:00  False  False  False  False   False  False  False  False   

symbol              XRPUSD                       
RSI_timeperiod          15            30         
RSI_is_below            40     50     40     50  
open_time                                        
2020-10-01 00:00:00  False  False  False  False  
2020-10-01 00:30:00  False  False  False  False  
2020-10-01 01:00:00  False  False  False  False  
2020-10-01 01:30:00  False  False  False  False  
2020-10-01 02:00:00  False  False  False  False  
...                    ...    ...    ...    ...  
2021-04-14 21:30:00  False  False  False  False  
2021-04-14 22:00:00  False  False  False  False  
2021-04-14 22:30:00  False  False  False  False  
2021-04-14 23:00:00  False  False  False  False  
2021-04-14 23:30:00  False  False  False  False  

[9408 rows x 20 columns]

In [2]:
strat_results_df, settings_results_df = sm.backtest_df(
    prices=price_data,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(7, 8.1, 1),
    risk_rewards=4,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=8,
    tsl_based_on=CandleBody.low,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=np.arange(3, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 4
Total order settings to test: 20
Total candles per symbol: 9,408
Total candles to test: 752,640

Total combinations to test: 80


In [4]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind.data[[('BTCUSDT', 15)]],
        "entries1": entries.iloc[:,[0]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=price_data.BTCUSDT,
    order_records=sm.select_row_and_simulate([0,20,19,14,15]),
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/


In [ ]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind.data[[('BTCUSDT', 15)]],
        "values2": rsi_ema_ind.data[[('BTCUSDT', 15)]],
        "entries1": entries.iloc[:,[0]],
    },
    "indicator2": {
        "values1": macd.data[[('BTCUSDT', 15)]],
        "entries1": entries.iloc[:,[0]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=price_data.BTCUSDT,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
